<a href="https://colab.research.google.com/github/raja-jamwal/blog-agentic-architectures/blob/main/Part_4_MCP%2C_Goal_Setting%2C_and_Exception_Handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

## MCP, Goal Setting, and Exception Handling

Welcome to Part 4 of the **Agentic Design Patterns** series.

We have built agents that can think (Part 2) and work in teams (Part 3). Now, we need to ensure they are **robust** and **interoperable**. A hobbyist agent works 80% of the time; a production agent must handle the other 20%—the errors, the timeouts, and the complex integrations.

We will cover:
1.  **Model Context Protocol (MCP):** The new standard for connecting AI to systems.
2.  **Goal Setting & Monitoring:** Keeping the agent focused on the objective.
3.  **Exception Handling:** Building "safety nets" for when things break.

### The Stack
*   **Python**
*   **LangChain**
*   **OpenAI** (GPT-4o-mini)

### The n8n Connection
*   **MCP** = Using the **HTTP Request Node** to query MCP servers, or using dedicated MCP nodes (if available in your version).
*   **Goal Setting** = Using a **Set Node** to define a `goal` variable, and passing it into a loop (using the **Split In Batches** or **Loop** nodes) until a condition is met.
*   **Exception Handling** = The **Error Trigger** node in n8n. If a node fails, the workflow jumps to a specific error-handling path (e.g., sending an alert or trying a different model).

In [1]:
# @title 1. Install Dependencies
!pip install -qU langchain langchain-openai langchain-core

import os
from getpass import getpass

# @title 2. Setup API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Environment Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 16.8 MB/s eta 0:00:00
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 10: Model Context Protocol (MCP)

**The Problem:** Every time you want to connect an LLM to a new database (Google Drive, Slack, Postgres), you have to write custom integration code. It's a mess of proprietary APIs.

**The Solution:** **MCP**. Think of it like "USB for AI." It's a standard way to expose data (Resources) and actions (Tools) so *any* agent can use them without custom code.

**The Scenario:** While we can't run a full MCP server inside Colab, we will simulate how an **MCP Client** (your agent) dynamically discovers and binds tools.

In [3]:
from langchain_core.tools import tool

# --- Simulation: The "MCP Server" ---
# Imagine this function lives on a remote server and is exposed via JSON-RPC.
# The MCP standard defines how the agent "discovers" this tool.

@tool
def mcp_filesystem_read(filepath: str) -> str:
    """MCP Tool: Reads a file from the secure sandbox."""
    # In a real scenario, this accesses the actual disk.
    mock_fs = {
        "config.json": '{"status": "active", "version": "1.0"}',
        "notes.txt": "Meeting at 2 PM."
    }
    return mock_fs.get(filepath, "Error: File not found.")

# --- The Client (Your Agent) ---
# The agent doesn't need to know HOW the tool works, just that it exists.
tools = [mcp_filesystem_read]

# Bind the discovered tools to the LLM
agent_with_mcp = llm.bind_tools(tools)

# --- Execution ---
query = "Read the configuration file 'config.json' and tell me the version."
response = agent_with_mcp.invoke(query)

print(f"Query: {query}")
print(f"Tool Call Generated: {response.tool_calls}")

# Note: In a real MCP setup, you would use an MCP Client library to
# automatically fetch these tools from a URL (e.g., `sse://localhost:8000`).

Query: Read the configuration file 'config.json' and tell me the version.
Tool Call Generated: [{'name': 'mcp_filesystem_read', 'args': {'filepath': 'config.json'}, 'id': 'call_0ZIf46jhnQDKqMaG5fmjAGl1', 'type': 'tool_call'}]
Query: Read the configuration file 'config.json' and tell me the version.
Tool Call Generated: [{'name': 'mcp_filesystem_read', 'args': {'filepath': 'config.json'}, 'id': 'call_ZtyblAcaHSW6Z0Pn7CfSSREm', 'type': 'tool_call'}]


## Pattern 11: Goal Setting and Monitoring (The Manager)

**The Problem:** Agents often get distracted. They might generate code that runs but doesn't actually solve the user's specific problem.

**The Solution:** **The Loop**. We set a clear Goal. The agent acts. Then, a "Monitor" (or Judge) checks if the goal was met. If not, the agent tries again.

**The Scenario:** We want the agent to write a specific Python function. We will loop until the code passes a check.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 1. The Worker (Doer) ---
worker_prompt = ChatPromptTemplate.from_template(
    "Write a Python function to achieve this goal: {goal}. Return ONLY code."
)
worker_chain = worker_prompt | llm | StrOutputParser()

# --- 2. The Monitor (Judge) ---
monitor_prompt = ChatPromptTemplate.from_template(
    """Goal: {goal}
    Code Generated: {code}

    Does this code accurately achieve the goal?
    Respond exactly with 'YES' or 'NO'."""
)
monitor_chain = monitor_prompt | llm | StrOutputParser()

# --- 3. The Loop ---
goal = "Calculate the Fibonacci sequence up to N"
max_retries = 3
attempt = 0
success = False

print(f"🎯 Goal: {goal}\n")

while attempt < max_retries and not success:
    attempt += 1
    print(f"--- Attempt {attempt} ---")

    # Generate
    code = worker_chain.invoke({"goal": goal})
    print(f"Generated Code Length: {len(code)} chars")

    # Monitor
    verdict = monitor_chain.invoke({"goal": goal, "code": code})
    print(f"Monitor Verdict: {verdict}")

    if "YES" in verdict.upper():
        success = True
        print("\n✅ Goal Achieved!")
        print(code)
    else:
        print("❌ Retrying...")

if not success:
    print("\n💀 Failed to achieve goal after max retries.")

🎯 Goal: Calculate the Fibonacci sequence up to N

--- Attempt 1 ---
Generated Code Length: 174 chars
Monitor Verdict: YES

✅ Goal Achieved!
```python
def fibonacci_up_to_n(n):
    fib_sequence = []
    a, b = 0, 1
    while a <= n:
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence
```


## Pattern 12: Exception Handling (The Safety Net)

**The Problem:** APIs go down. Models get rate-limited. Inputs are malformed. If your script crashes, your user is unhappy.

**The Solution:** **Fallbacks**. If the primary model fails, automatically switch to a backup model or a simpler logic path.

**The Scenario:** We will simulate a broken model (using a fake model name) and demonstrate how LangChain automatically falls back to a working model.

In [5]:
from langchain_openai import ChatOpenAI

# --- 1. Define a Broken Model ---
# This model name does not exist, so it will raise an error.
broken_llm = ChatOpenAI(model="gpt-fake-model-v99")

# --- 2. Define the Backup Model ---
# This is our reliable worker.
backup_llm = ChatOpenAI(model="gpt-4o-mini")

# --- 3. Create the Fallback Chain ---
# We try the broken one first. If it fails, we use the backup.
# .with_fallbacks() is a powerful LangChain feature.
resilient_llm = broken_llm.with_fallbacks([backup_llm])

# --- Execution ---
try:
    print("Attempting to invoke chain...")
    # This will internally try 'gpt-fake-model', fail, catch the error,
    # and seamlessly switch to 'gpt-4o-mini'.
    response = resilient_llm.invoke("Say 'Hello, I am the backup!'")
    print(f"\nSUCCESS: {response.content}")
except Exception as e:
    print(f"CRITICAL FAILURE: {e}")

# Note: This is crucial for production. You can fallback from GPT-4 (expensive)
# to GPT-3.5 (cheap) or from OpenAI to Anthropic (different provider).

Attempting to invoke chain...

SUCCESS: Hello, I am the backup!


## Summary

You have now built the infrastructure for a professional agent:

1.  **MCP:** You understand how to standardize tool connections.
2.  **Goal Setting:** You created a loop that ensures quality before finishing.
3.  **Exception Handling:** You built a system that doesn't crash when errors occur.

In **Part 5**, we will focus on **Reliability**. We will bring humans into the loop and connect our agents to vast knowledge bases using **RAG**.